In [4]:
!pip install langchain chromadb faiss-cpu openai tiktoken langchain_openai langchain-community wikipedia

  Using cached chromadb-1.0.20-cp39-abi3-win_amd64.whl.metadata (7.4 kB)
  Using cached faiss_cpu-1.12.0-cp313-cp313-win_amd64.whl.metadata (5.2 kB)
  Using cached build-1.3.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached pybase64-1.4.2-cp313-cp313-win_amd64.whl.metadata (9.0 kB)
  Using cached uvicorn-0.35.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached posthog-5.4.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached onnxruntime-1.22.1-cp313-cp313-win_amd64.whl.metadata (5.1 kB)
  Using cached opentelemetry_api-1.36.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.36.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached opentelemetry_sdk-1.36.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached pypika-0.48.9-py2.py3-none-any.whl
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using c


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Wikipedia Retriever

In [5]:
from langchain_community.retrievers import WikipediaRetriever

In [6]:
# Initialize the retriever (optional: set language and top_k)
retriever = WikipediaRetriever(top_k_results=2, lang="en")

In [7]:

# Define your query
query = "the geopolitical history of india and pakistan from the perspective of a chinese"

# Get relevant Wikipedia documents
docs = retriever.invoke(query)

In [8]:
docs

[Document(metadata={'title': 'China–India relations', 'summary': "China and India maintained peaceful relations for thousands of years, but their relationship has varied since the Chinese Communist Party (CCP)'s victory in the Chinese Civil War in 1949 and the annexation of Tibet by the People's Republic of China. The two nations have sought economic cooperation with each other, while frequent border disputes and economic nationalism in both countries are major points of contention.\nCultural and economic relations between China and India date back to ancient times. The Silk Road not only served as a major trade route between India and China, but is also credited for facilitating the spread of Buddhism from India to East Asia. During the 19th century, China was involved in a growing opium trade with the East India Company, which exported opium grown in India. During World War II, both British India and the Republic of China (ROC) played a crucial role in halting the progress of Imperia

In [9]:
# Print retrieved content
for i, doc in enumerate(docs):
    print(f"\n--- Result {i+1} ---")
    print(f"Content:\n{doc.page_content}...")  # truncate for display


--- Result 1 ---
Content:
China and India maintained peaceful relations for thousands of years, but their relationship has varied since the Chinese Communist Party (CCP)'s victory in the Chinese Civil War in 1949 and the annexation of Tibet by the People's Republic of China. The two nations have sought economic cooperation with each other, while frequent border disputes and economic nationalism in both countries are major points of contention.
Cultural and economic relations between China and India date back to ancient times. The Silk Road not only served as a major trade route between India and China, but is also credited for facilitating the spread of Buddhism from India to East Asia. During the 19th century, China was involved in a growing opium trade with the East India Company, which exported opium grown in India. During World War II, both British India and the Republic of China (ROC) played a crucial role in halting the progress of Imperial Japan. After India became independent 

## Vector Store Retriever

In [11]:
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_core.documents import Document

In [12]:
embedding_model = OllamaEmbeddings(model="nomic-embed-text:latest")

In [13]:
# Step 1: Your source documents
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [14]:
# Step 2: Initialize embedding model
#embedding_model = OpenAIEmbeddings()

# Step 3: Create Chroma vector store in memory
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name="my_collection"
)

In [15]:
# Step 4: Convert vectorstore into a retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [16]:
query = "What is Chroma used for?"
results = retriever.invoke(query)

In [17]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Chroma is a vector database optimized for LLM-based search.

--- Result 2 ---
Embeddings convert text into high-dimensional vectors.


In [18]:
results = vectorstore.similarity_search(query, k=2)

In [19]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Chroma is a vector database optimized for LLM-based search.

--- Result 2 ---
Embeddings convert text into high-dimensional vectors.


## MMR

In [20]:
# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [21]:
from langchain_community.vectorstores import FAISS

# Initialize OpenAI embeddings
#embedding_model = OpenAIEmbeddings()

# Step 2: Create the FAISS vector store from documents
vectorstore = FAISS.from_documents(
    documents=docs,
    embedding=embedding_model
)

In [22]:
# Enable MMR in the retriever
retriever = vectorstore.as_retriever(
    search_type="mmr",                   # <-- This enables MMR
    search_kwargs={"k": 3, "lambda_mult": 0.5}  # k = top results, lambda_mult = relevance-diversity balance
)

In [23]:
query = "What is langchain?"
results = retriever.invoke(query)

In [ ]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
LangChain is used to build LLM based applications.

--- Result 2 ---
Embeddings are vector representations of text.

--- Result 3 ---
LangChain supports Chroma, FAISS, Pinecone, and more.


## Multiquery Retriever

In [24]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever

In [25]:
# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [26]:
# Initialize OpenAI embeddings
#embedding_model = OpenAIEmbeddings()

# Create FAISS vector store
vectorstore = FAISS.from_documents(documents=all_docs, embedding=embedding_model)

In [27]:
# Create retrievers
similarity_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [28]:
multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
    llm=ChatOpenAI(model="gpt-3.5-turbo")
)

In [29]:
# Query
query = "How to improve energy levels and maintain balance?"

In [30]:
# Retrieve results
similarity_results = similarity_retriever.invoke(query)
multiquery_results= multiquery_retriever.invoke(query)

In [31]:
for i, doc in enumerate(similarity_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

print("*"*150)

for i, doc in enumerate(multiquery_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
The solar energy system in modern homes helps balance electricity demand.

--- Result 3 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 4 ---
Photosynthesis enables plants to produce energy by converting sunlight.

--- Result 5 ---
Consuming leafy greens and fruits helps detox the body and improve longevity.
******************************************************************************************************************************************************

--- Result 1 ---
The solar energy system in modern homes helps balance electricity demand.

--- Result 2 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 3 ---
Consuming leafy greens and fruits helps detox the body and improve longevity.

--- Result 4 ---
Photosynthesis enables plants to produce energy by converting sun

## ContextualCompressionRetriever

In [32]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.documents import Document

In [33]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [34]:
# Create a FAISS vector store from the documents
#embedding_model = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embedding_model)

In [35]:
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [48]:
# Set up the compressor using an LLM
from langchain_ollama import ChatOllama
model= ChatOllama(model="deepseek-r1:1.5b")
compressor = LLMChainExtractor.from_llm(model)

In [49]:
# Create the contextual compression retriever
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [50]:
# Query the retriever
query = "What is photosynthesis?"
compressed_results = compression_retriever.invoke(query)

In [51]:
for i, doc in enumerate(compressed_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)



--- Result 1 ---
<think>
Okay, so I need to figure out what photosynthesis is based on the given context and question. Let me read through the context again carefully.

The context says that the Grand Canyon is a natural wonder visited by tourists. It mentions that photosynthesis is the process green plants use to convert sunlight into energy. It also notes how many people visit each year because of this activity, with millions of visitors every year. Additionally, it talks about how the rocks in the Grand Canyon date back millions of years.

The question is asking specifically for a definition of photosynthesis. So I need to find a part of the context that directly relates to that topic. Looking through the context, there's an exact sentence: "Photosynthesis is the process by which green plants convert sunlight into energy." That clearly defines what photosynthesis is.

But wait, let me make sure none of the other parts are relevant or could be used for this question. The extracted p